In [1]:
#pip install spacy

In [2]:
#pip install -U pip setuptools wheel
#!pip install -U spacy
#!python -m spacy download en_core_web_sm

In [3]:
import numpy as np
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn import metrics

import string
import spacy
np.random.seed(42)

In [4]:
data = pd.read_csv("D:\\Data\\NLP\\pradip\\toxic_commnets.csv",on_bad_lines='error',skiprows = range(1000, 159570), engine="python")
data.tail()

,comment_text,toxic
996,It is common knowledge that Karaims (but not K...,0
997,", 12 April 2006 (UTC)\nThen rewrite and expand...",0
998,"""I was trying to inject some humour (as eviden...",0
999,And it looks like it was actually you who put ...,0
1000,"""\nAnd ... I really don't think you understand...",0


In [5]:
# Spacy Model

In [6]:
nlp = spacy.load("en_core_web_sm")

In [7]:
# Stop Words

In [8]:
stop_words = nlp.Defaults.stop_words
print(stop_words)

{'except', 'whatever', 're', 'then', 'however', 'front', 'does', 'forty', "'d", 'together', 'might', 'eleven', 'ours', 'nine', 'really', 'now', 'but', 'part', 'across', 'nowhere', 'whenever', 'seemed', 'very', 'itself', 'without', 'myself', 'thus', 'during', 'indeed', 'me', 'their', 'just', 'the', 'hereupon', 'become', 'hereby', 'yours', 'nobody', 'for', 'must', 'often', 'our', 'meanwhile', 'here', 'whoever', 'herself', 'n’t', 'doing', 'or', 'take', 'something', 'toward', 'give', 'amongst', 'made', 'last', 'make', 'first', 'a', 'nor', 'due', 'four', 'seems', 'under', 'otherwise', 'anyone', 'ca', 'seeming', 'nevertheless', 'although', 'when', 'you', 'my', 'namely', 'becomes', 'few', 'seem', 'would', 'even', 'can', 'am', 'back', 'his', 'some', 'all', 'wherever', 'amount', 'everyone', 'could', 'himself', 'eight', 'everything', 'go', 'other', 'with', 'this', 'anywhere', 'serious', 'via', 'per', 'top', 'whereafter', '’ll', 'so', 'please', 'him', 'third', 'herein', 'own', 'beforehand', 'who'

In [9]:
# punctuations

In [10]:
punctuations = string.punctuation
print(punctuations)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [11]:
# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    doc = nlp(sentence)



    # print(doc)
    # print(type(doc))

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() for word in doc ]

    # print(mytokens)

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [12]:
sentence = "I am eating apple, I like apple ?"
spacy_tokenizer(sentence)

['eat', 'apple', 'like', 'apple']

## Count Vectoriser

In [13]:
count_vector = CountVectorizer(tokenizer = spacy_tokenizer)

In [14]:
count_vector.fit_transform(["I am eating apple, I like apple","I am playing cricket"]).toarray() 

C:\Users\intel\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


array([[2, 0, 1, 1, 0],
       [0, 1, 0, 0, 1]], dtype=int64)

In [15]:
count_vector.get_feature_names_out()

array(['apple', 'cricket', 'eat', 'like', 'play'], dtype=object)

In [16]:
count_vector.vocabulary_

{'eat': 2, 'apple': 0, 'like': 3, 'play': 4, 'cricket': 1}

In [17]:
from sklearn.model_selection import train_test_split

X = data['comment_text'] # the features we want to analyze
ylabels = data['toxic'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2,stratify=ylabels)
     

In [18]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

In [19]:
X_train_vetcors= count_vector.fit_transform(X_train)
X_test_vetcors= count_vector.transform(X_test)

C:\Users\intel\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [20]:
classifier.fit(X_train_vetcors,y_train)

LogisticRegression()

In [21]:
predicted = classifier.predict(X_test_vetcors)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.9203980099502488
Logistic Regression Precision: 0.8571428571428571
Logistic Regression Recall: 0.2857142857142857


## TFIDF Vectoriser

In [22]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [23]:
tfidf_vector.fit_transform(["I am eating apple, I like apple","I am playing cricket"]).toarray() 

C:\Users\intel\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


array([[0.81649658, 0.        , 0.40824829, 0.40824829, 0.        ],
       [0.        , 0.70710678, 0.        , 0.        , 0.70710678]])

In [24]:
from sklearn.model_selection import train_test_split

X = data['comment_text'] # the features we want to analyze
ylabels = data['toxic'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2,stratify=ylabels)

In [25]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

In [26]:
X_train_vetcors= tfidf_vector.fit_transform(X_train)
X_test_vetcors= tfidf_vector.transform(X_test)

C:\Users\intel\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [27]:
classifier.fit(X_train_vetcors,y_train)

LogisticRegression()

In [28]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)
X_train_vetcors= tfidf_vector.fit_transform(X_train)
X_test_vetcors= tfidf_vector.transform(X_test)

C:\Users\intel\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [29]:
classifier = LogisticRegression()
classifier.fit(X_train_vetcors,y_train)
predicted = classifier.predict(X_test_vetcors)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.9104477611940298
Logistic Regression Precision: 1.0
Logistic Regression Recall: 0.14285714285714285
